In [1]:
#TODO test notebook on workshop computer
#     gather results into report (best yed layout to use?), tabulate results of exercises 7-8 
#     consider removing merging of 0 distance vertices, code is messy
#     find more optimal solutions for subgraphs.py if time permits
# Questions
#     Does question 1c require the MST using the results of 1a or 1b? A: choose whatever
#     For graph creation, the jaccard measure shouldn't be used directly? Using matrix of (1 - j) values for now
        # Make this (1 - j) occur at matrix creation rather than just before graphs are generated
        # A: use whatever is in the cited source
#     Announcement suggests merging vertices with distance of 0, is this necessary or optional?
        # This might change the results of the KNN algorithm
        # A: Optional

In [2]:
# Cite jaccad measure and jaccard measure complement (distance) resource (Wikipedia is acceptable)
# Cite hamming distance resource
# Cite lecture notes for graph algorithms
# Section of report containing code, python version and project setup (submission only accepts pdf, can't .py files)

In [3]:
import math
import networkx as nx
import numpy as np
import os
import pandas as pd
import pylab as plt

In [4]:
data = pd.read_csv("USPresidency.csv")
print(data.head())

   Year  Q1  Q2  Q3  Q4  Q5  Q6  Q7  Q8  Q9  Q10  Q11  Q12  Target
0  1864   0   0   0   0   1   0   0   1   1    0    0    0       1
1  1868   1   1   0   0   0   0   1   1   1    0    1    0       1
2  1872   1   1   0   0   1   0   1   0   0    0    1    0       1
3  1880   1   0   0   1   0   0   1   1   0    0    0    0       1
4  1888   0   0   0   0   1   0   0   0   0    0    0    0       1


In [5]:
# Store election years to be used as labels
elections_data = data.iloc[:, 0]
elections = []
for index, election in enumerate(elections_data):
    # Convert from int64 to str (necessary to prevent networkx type errors)
    elections.append(str(elections_data[index]))
    print(election, end=", ")

1864, 1868, 1872, 1880, 1888, 1900, 1904, 1908, 1916, 1924, 1928, 1936, 1940, 1944, 1948, 1956, 1964, 1972, 1860, 1876, 1884, 1892, 1896, 1912, 1920, 1932, 1952, 1960, 1968, 1976, 1980, 

In [6]:
# Remove class type and instance label
data = data.iloc[:, 1:-1]
print(data.head())

   Q1  Q2  Q3  Q4  Q5  Q6  Q7  Q8  Q9  Q10  Q11  Q12
0   0   0   0   0   1   0   0   1   1    0    0    0
1   1   1   0   0   0   0   1   1   1    0    1    0
2   1   1   0   0   1   0   1   0   0    0    1    0
3   1   0   0   1   0   0   1   1   0    0    0    0
4   0   0   0   0   1   0   0   0   0    0    0    0


In [7]:
# Functions used for generating required distance matrices
def hamming_distance(x, y):
    distance = 0
    for i in range(len(x)):
        if x[i] != y[i]:
            distance += 1
    
    return distance


def union_size(x, y):
    count = 0
    for index, item in enumerate(x):
        if item or y[index] == 1:
            count += 1
    
    return count


def intersect_size(x, y):
    count = 0
    for index, item in enumerate(x):
        if item == y[index] and item == 1:
            count += 1
            
    return count

In [8]:
print(data.iloc[0, :].values)

[0 0 0 0 1 0 0 1 1 0 0 0]


In [9]:
def merge_dupes(D):
    merge = True
    merged = []
    # End the loop after a full pass of the matrix without any merges occurring
    while merge:
        cur_j = 0
        merge = False
        # Iterate through non-diagonal entries
        for i in range(len(D)):
            for j in range(i + 1, len(D)):
                if D[i][j] == 0:
                    cur_i = i
                    cur_j = j
                    
        
        if cur_j != 0:
            D = np.delete(D, cur_j, 0)
            D = np.delete(D, cur_j, 1)
            merged.append([cur_i, cur_j])
            merge = True
            
    return D, merged

In [10]:
# Create Hamming Distance and Jaccard measure matrices and save as csv
ninst, nattr = data.shape
HD = np.zeros((ninst, ninst))
JD = np.zeros((ninst, ninst))

for i in range(ninst):
    idata = data.iloc[i, :].values
    for j in range(ninst):
        jdata = data.iloc[j, :].values
        
        # Hamming Distance matrix
        dist = hamming_distance(idata, jdata)
        HD[i][j]= dist
        
        # Jaccard measure matrix
        jaccard = intersect_size(idata, jdata) / union_size(idata, jdata)
        # Take the complement of the Jaccard measure to get a measure of dissimilarity
        JD[i][j] = 1 - jaccard
        

if os.path.exists("US"):
    print("Path exists")
else:
    print("Path not found, creating directory")
    os.mkdir("US")

HD, merged  = merge_dupes(HD)
JD, _ = merge_dupes(JD)

np.savetxt("US/HammingDistanceMatrix.csv", HD, fmt="%.2f", delimiter=",")
np.savetxt("US/JaccardMeasureMatrix.csv", JD, fmt="%.2f", delimiter=",")

Path exists


In [11]:
# Update labels to reflect merges
for indices in merged:
    elections[indices[0]] = elections[indices[0]] + "," + elections[indices[1]]
    elections = np.delete(elections, indices[1])

print(elections)

['1864,1972' '1868' '1872,1904' '1880' '1888' '1900' '1908' '1916' '1924'
 '1928' '1936' '1940' '1944' '1948' '1956' '1964' '1860' '1876' '1884'
 '1892' '1896' '1912' '1920' '1932' '1952' '1960' '1968' '1976' '1980']


In [12]:
# Create the same matrices but for the columns(attributes) rather than the rows(instances)
HD_f = np.zeros((nattr, nattr))
JD_f = np.zeros((nattr, nattr))

for i in range(nattr):
    idata = data.iloc[:, i].values
    for j in range(nattr):
        jdata = data.iloc[:, j].values
        
        # Hamming Distance matrix
        dist = hamming_distance(idata, jdata)
        HD_f[i][j] = dist
        
        # Jaccard measure matrix
        jaccard = intersect_size(idata, jdata) / union_size(idata, jdata)
        JD_f[i][j] = 1 - jaccard
        
        
np.savetxt("US/AttributeHammingDistanceMatrix.csv", HD_f, fmt="%.2f", delimiter=",")
np.savetxt("US/AttributeJaccardMeasureMatrix.csv", JD_f, fmt="%.2f", delimiter=",")

In [13]:
# Generate set of vertices
V = set(n for n in range(len(HD)))

In [14]:
import subgraphs as sg

In [15]:
# Used to reload library in Jupyter after it has been edited
import importlib
importlib.reload(sg)
print("Subgraphs module version ", sg.__version__)

Subgraphs module version  0.3.1


In [16]:
print("Generating graphs and saving as .gml")

Generating graphs and saving as .gml


In [17]:
# Create MST using Hamming Distance matrix
mst_hd = sg.mst(V, HD, "US/us-graph-mst-hd.gml", elections)

In [18]:
# Create RNG using Hamming Distance matrix
_ = sg.rngraph(V, HD, "US/us-graph-rng-hd.gml", elections)

In [19]:
# Create MST using Jaccard distance matrix
mst_jd = sg.mst(V, JD, "US/us-graph-mst-jd.gml", elections)

In [20]:
# Create RNG using Jaccard distance matrix
_ = sg.rngraph(V, JD, "US/us-graph-rng-jd.gml", elections)

In [21]:
# Create KNN with k=2 using Hamming Distance matrix
knn_hd = sg.knn(V, HD, 2, "US/us-graph-knn-hd.gml", elections)

In [22]:
# Create KNN with k=2 using Jaccard distance matrix
knn_jd = sg.knn(V, JD, 2, "US/us-graph-knn-jd.gml", elections)

In [23]:
# MST-KNN graph, k=2, Hamming Distance
_ = sg.mst_knn(mst_hd, knn_hd, "US/us-graph-mst-knn-hd.gml")

In [24]:
# MST-KNN graph, k=2, Jaccard distance
_ = sg.mst_knn(mst_jd, knn_jd, "US/us-graph-mst-knn-jd.gml")

In [25]:
print("Done")

Done


In [26]:
# Generate MST-KNN using networkx's mst function to compare output
G = nx.Graph()

labels = {}
for n in range(len(HD)):
    for m in range(len(HD) - (n+1)):
        u = elections[n]
        v = elections[n+m+1]
        G.add_edge(u, v, weight=HD[n][n+m+1])
        labels[(u, v)] = "{:.2f}".format(HD[n][n+m+1])


mst_nx_edges = nx.algorithms.tree.mst.prim_mst_edges(G, True)

mst_nx = []
print(knn_hd[0])
for edge in mst_nx_edges:
    mst_nx.append([edge[0], edge[1], -1]) # Filler weight value, clusters are important due to shared edges not weights
    
_ = sg.mst_knn(mst_nx, knn_hd, "US/networkx-mst-knn-hd.gml")

['1864,1972', '1916', 1.0]


In [27]:
G = nx.Graph()

labels = {}
for n in range(len(JD)):
    for m in range(len(JD) - (n+1)):
        u = elections[n]
        v = elections[n+m+1]
        G.add_edge(u, v, weight=JD[n][n+m+1])
        labels[(u, v)] = "{:.2f}".format(JD[n][n+m+1])


mst_nx_edges = nx.algorithms.tree.mst.prim_mst_edges(G, True)

mst_nx = []
print(knn_hd[0])
for edge in mst_nx_edges:
    mst_nx.append([edge[0], edge[1], -1]) # Filler weight value, clusters are important due to shared edges not weights
    
_ = sg.mst_knn(mst_nx, knn_jd, "US/networkx-mst-knn-jd.gml")

['1864,1972', '1916', 1.0]
